In [1]:
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd


# Step 1 - Scraping (NASA Mars News, JPL Mars Space Images - Featured Image, Mars Weather, Mars Facts, Mars Hemisperes)

In [2]:
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

mars_weather_url = "https://twitter.com/marswxreport?lang=en"

mars_facts_url = "https://space-facts.com/mars/"

hemisphere_image_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"





## Gather Mars News

In [3]:
response_nasa = requests.get(nasa_url)

soup_nasa = bs(response_nasa.text, 'html.parser')
# print(soup_nasa.prettify())

In [4]:
news_title = soup_nasa.find('div', 'content_title', 'a').get_text()
print(news_title)



NASA Invests in Visionary Technology 




In [5]:
news_p = soup_nasa.find('div', 'rollover_description_inner').text
print(news_p)


NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.



## Use Splinter to Capture JPL Image

In [6]:
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
browser.visit(featured_image_url)
time.sleep(2)

browser.click_link_by_id('full_image')
time.sleep(2)

browser.click_link_by_partial_text('more info')
time.sleep(2)

xpath = '//figure[@class="lede"]'
browser.find_by_xpath(xpath).click()
time.sleep(2)

featured_image_jpg = browser.url

In [8]:
print(featured_image_jpg)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15256_hires.jpg


## Find Current Weather on Mars

In [9]:
import tweepy
import json
import numpy as np
import pandas as pd

In [10]:
consumer_key = "E7dUfrmzrNI0Ed8GhjP7RXvE1"
consumer_secret = "p2aWWu1fUZ05Dx7VMXM0TLWY6eL1q7O3WzlsieuJPZ8KjxJHlq"
access_token = "355054202-sAmlB68y35O78i9qYplRE95G7eEpLisaewU0p31N"
access_token_secret = "p6P9aH8tVMsIttvN1mzJes5G3Xtr8ep55BDlShnoD8Lla"

In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [12]:
target_user = '@MarsWxReport'

In [13]:
tweet = api.user_timeline(target_user, count=1)[0]

tweet_text = json.dumps(tweet._json, indent=3)
tweet = json.loads(tweet_text)
mars_weather = tweet['text']

mars_weather

'Sol 2026 (April 18, 2018), Sunny, high -6C/21F, low -73C/-99F, pressure at 7.19 hPa, daylight 05:26-17:21'

## Collect Mars Facts

In [14]:
tables = pd.read_html(mars_facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [15]:
df = tables[0]
df.columns = ['','Values']
# df.head()

In [16]:
df.set_index('',inplace=True)
df.head()

,Values
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [17]:
html_table = df.to_html()
# html_table

In [18]:
html_table.replace('\n','')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Images of Mars Hemispheres

In [29]:
browser.visit(hemisphere_image_url)
html = browser.html
soup_hemispheres = bs(html, 'html.parser')

hemisphere_img_urls = []

products = soup_hemispheres.find("div", class_ = "result-list")
hemispheres = products.find_all("div", class_ = "item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    link_ref = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + link_ref  
    browser.visit(image_link)
    html = browser.html
    soup_hemispheres=bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_img_urls.append({"title": title, "img_url": image_url})

In [30]:
hemisphere_img_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]